In [1]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print("Physical GPUs: {}, Logical GPUs: {}".format(len(gpus), len(logical_gpus)))
else:
    print("CPU only")

x = np.arange(-1, 1, 0.0001)
y = 0.8 * x + 0.2

model = tf.keras.Sequential([tf.keras.layers.Dense(1, activation=None)])
model.compile("sgd", "mse")
model.build(input_shape=(0,1))
model.summary()
model.fit(x, y, epochs=5)

print("ground truth: 0.8, 0.2")
print("estimated: ", model.variables[0][0,0].numpy(), model.variables[1][0].numpy())


2025-10-16 01:33:51.608398: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.13.0
Physical GPUs: 2, Logical GPUs: 2


2025-10-16 01:33:54.930357: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11287 MB memory:  -> device: 0, name: NVIDIA TITAN X (Pascal), pci bus id: 0000:03:00.0, compute capability: 6.1
2025-10-16 01:33:54.931672: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 11549 MB memory:  -> device: 1, name: NVIDIA TITAN X (Pascal), pci bus id: 0000:04:00.0, compute capability: 6.1


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (0, 1)                    2         
                                                                 
Total params: 2 (8.00 Byte)
Trainable params: 2 (8.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5


2025-10-16 01:33:55.736580: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f5300a61ab0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-16 01:33:55.736616: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2025-10-16 01:33:55.736623: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2025-10-16 01:33:55.812739: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


 73/625 [==>...........................] - ETA: 1s - loss: 0.1318

2025-10-16 01:33:56.111618: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 [==============================] - 2s 2ms/step - loss: 0.0238
Epoch 2/5
625/625 [==============================] - 1s 2ms/step - loss: 5.1882e-06
Epoch 3/5
625/625 [==============================] - 1s 2ms/step - loss: 1.2134e-09
Epoch 4/5
625/625 [==============================] - 1s 2ms/step - loss: 3.6031e-12
Epoch 5/5
625/625 [==============================] - 2s 2ms/step - loss: 3.2314e-12
ground truth: 0.8, 0.2
estimated:  0.8000031 0.20000002


In [35]:
tf.test.is_gpu_available()

2025-09-19 08:12:52.671010: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:0 with 115 MB memory:  -> device: 0, name: NVIDIA TITAN X (Pascal), pci bus id: 0000:03:00.0, compute capability: 6.1
2025-09-19 08:12:52.671480: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:1 with 11413 MB memory:  -> device: 1, name: NVIDIA TITAN X (Pascal), pci bus id: 0000:04:00.0, compute capability: 6.1


True

In [28]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

In [29]:
# --- 1. データ準備 (前回と同じ) ---
df = pd.read_excel("./深層学習.xlsx")
X = df[['left_x', 'left_y', 'right_x', 'right_y', 'left_pupil', 'right_pupil']]
y = df["is_correct"]


In [32]:
%pip list

Package                      Version
---------------------------- --------------------
absl-py                      1.4.0
asttokens                    3.0.0
astunparse                   1.6.3
backcall                     0.2.0
cachetools                   5.3.1
certifi                      2019.11.28
chardet                      3.0.4
comm                         0.2.3
contourpy                    1.1.1
cycler                       0.12.1
dbus-python                  1.2.16
debugpy                      1.8.17
decorator                    5.2.1
et_xmlfile                   2.0.0
executing                    2.2.1
filelock                     3.16.1
flatbuffers                  23.5.26
fonttools                    4.57.0
fsspec                       2025.3.0
gast                         0.4.0
google-auth                  2.21.0
google-auth-oauthlib         1.0.0
google-pasta                 0.2.0
grpcio                       1.56.0
h5py                         3.9.0
idna                 

In [31]:
# X, y = make_classification(
#     n_samples=1000,
#     n_features=10,
#     n_informative=4,
#     n_redundant=2,
#     n_classes=2,
#     random_state=42
# )
# feature_names = [f'feature_{i}' for i in range(10)]
# X = pd.DataFrame(X, columns=feature_names)

feature_names = X.columns.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# --- 2. モデル構築 (アテンション可視化対応) ---

# Transformerブロックのcallメソッドを変更し、アテンションスコアを返せるようにする
# ...existing code...
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim)]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training=None, return_attention_scores=False):  # ← ここを修正
        attention_output, attention_scores = self.att(
            inputs, inputs, return_attention_scores=True
        )
        attention_output = self.dropout1(attention_output, training=training)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        output = self.layernorm2(out1 + ffn_output)
        if return_attention_scores:
            return output, attention_scores
        return output
# ...existing code...

# モデル定義関数を修正
def create_visualizable_transformer(num_features, embed_dim=32, num_heads=4, ff_dim=32):
    inputs = tf.keras.layers.Input(shape=(num_features,))
    
    # 1. 各特徴量を独立したトークンとして扱うため、次元を追加
    # (Batch, Features) -> (Batch, Features, 1)
    x = tf.keras.layers.Reshape((num_features, 1))(inputs)

    # 2. 各特徴量を個別に埋め込みベクトルに変換
    # Conv1Dは各タイムステップ(今回は特徴量)に同じカーネルを適用するため、
    # TimeDistributed(Dense) と同様の役割を果たし、効率的
    x = tf.keras.layers.Conv1D(embed_dim, kernel_size=1, activation='relu')(x)

    # 3. Transformerブロックを適用し、アテンションスコアも受け取る
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x, attention_scores = transformer_block(x, return_attention_scores=True)
    
    # 4. 分類のための後処理
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(20, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid", name="classification_output")(x)

    # 最終的な分類出力と、途中のアテンションスコアを両方出力するモデルを作成
    model = tf.keras.Model(
        inputs=inputs,
        outputs=[outputs, attention_scores]
    )
    return model

# モデルのインスタンスを作成
model = create_visualizable_transformer(num_features=X_train.shape[1])

# モデルのコンパイル (出力が2つあるため、lossも2つ指定)
# アテンションスコアには損失は不要なので、Noneを指定
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=["binary_crossentropy", None], # attention_scoresのlossは計算しない
    metrics={"classification_output": "accuracy"}
)

model.summary()


# --- 3. 学習 (前回と同じ) ---
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=20,
    validation_split=0.2,
    verbose=0 # 学習過程の表示をオフに
)
print("学習が完了しました。")


# --- 4. アテンションの可視化 ---

# テストデータから1つサンプルを選んで予測
sample_index = 0
sample_input = np.expand_dims(X_test[sample_index], axis=0)

# 予測と同時にアテンションスコアを取得
prediction, attention_scores = model.predict(sample_input)

# アテンションスコアの整形
# (Batch, Heads, Features, Features) -> (Heads, Features, Features)
attention_scores = attention_scores[0]
# ヘッド間で平均を取る
attention_matrix = np.mean(attention_scores, axis=0)

# ヒートマップで可視化
plt.figure(figsize=(10, 8))
sns.heatmap(attention_matrix, xticklabels=feature_names, yticklabels=feature_names, annot=True, cmap='viridis')
plt.title(f'Attention Matrix for Sample #{sample_index}')
plt.xlabel('Key (attended to)')
plt.ylabel('Query (attending from)')
plt.show()

print(f"\nサンプル #{sample_index} の予測確率: {prediction[0][0]:.4f}")
print(f"正解ラベル: {y_test[sample_index]}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 6)]               0         
                                                                 
 reshape_1 (Reshape)         (None, 6, 1)              0         
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 32)             64        
                                                                 
 transformer_block_1 (Trans  ((None, 6, 32),           19040     
 formerBlock)                 (None, 4, 6, 6))                   
                                                                 
 global_average_pooling1d_1  (None, 32)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_6 (Dropout)         (None, 32)                0   

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'pandas.core.series.Series'>]